In [1]:
import psycopg2
from pandas.io import sql as pd_sql

In [2]:
con = psycopg2.connect(host = 'localhost',
                      database = 'applause',
                      user = 'JR',
                       port = 5432)

### Two Cursor Examples

In [3]:
cur = con.cursor() # Create a cursor

In [4]:
# The total bugs found for every tester that has ever tested a device
cur.execute("""SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM testers 
INNER JOIN bugs
ON testers.testerId = bugs.testerId
GROUP BY  firstName, lastName
ORDER BY COUNT(bugId) DESC;""")

rows = cur.fetchall()

for row in rows:
    print(row)
    
cur.execute('rollback;') # reset the cursor for a future query

('Taybin', 'Rutkin', 125)
('Lucas', 'Lowry', 117)
('Sean', 'Wellington', 116)
('Miguel', 'Bautista', 114)
('Stanley', 'Chen', 110)
('Mingquan', 'Zheng', 109)
('Leonard', 'Sutton', 106)
('Darshini', 'Thiagarajan', 104)
('Michael', 'Lubavin', 99)


In [5]:
# The total bugs found for each tester when given specific devices but withou specifing countries
# This example specifies the devices: iPhone 4 and iPhone 5
cur.execute("""SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM bugs 
INNER JOIN testers
ON bugs.testerid = testers.testerid
INNER JOIN devices
ON bugs.deviceId = devices.deviceId
WHERE deviceName IN ('iPhone 4', 'iPhone 5')
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;""")

rows = cur.fetchall()
for row in rows:
    print(row)

cur.execute('rollback;')

('Stanley', 'Chen', 110)
('Taybin', 'Rutkin', 66)
('Sean', 'Wellington', 58)
('Miguel', 'Bautista', 53)
('Leonard', 'Sutton', 32)
('Mingquan', 'Zheng', 21)


In [6]:
cur.close() # Close the cursor

### Two Pandas SQL Quieries

In [7]:
# The total bugs found for each person given specific countries but without specifing specific devices
# This example specifies the countries: US and JP
    
query3 = """SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM testers
INNER JOIN bugs
ON testers.testerId = bugs.testerId
WHERE country IN ('US', 'JP')
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;"""


country_specified_query = pd_sql.read_sql(query3, con)

country_specified_query.head()

,firstname,lastname,bugs_found
0,Taybin,Rutkin,125
1,Lucas,Lowry,117
2,Sean,Wellington,116
3,Miguel,Bautista,114
4,Mingquan,Zheng,109


In [8]:
# The total bugs for each tester given specific countries and specific devices
# This example specifices the countries: US and JP; The example specifices the devices: iPhone 4 and iPhone 5

query4 = """SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM bugs
INNER JOIN testers
ON bugs.testerId = testers.testerId
INNER JOIN devices
ON bugs.deviceId = devices.deviceId
WHERE country IN ('US', 'JP')  AND devicename IN ('iPhone 4', 'iPhone 5')
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;"""

country_and_devices_specified_query = pd_sql.read_sql(query4, con)

country_and_devices_specified_query.head()

,firstname,lastname,bugs_found
0,Taybin,Rutkin,66
1,Sean,Wellington,58
2,Miguel,Bautista,53
3,Mingquan,Zheng,21


In [9]:
con.close() # Close the connection